In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
#顯示所有列
pd.set_option('display.max_columns', None)

In [3]:
# Reading the data
train = pd.read_csv('tbrain_cc_training_48tags_hash_final.csv')
predict_test = pd.read_csv('predict_test.csv')
print(train.shape)
print(train.info())
train.head(5)

(32975653, 53)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32975653 entries, 0 to 32975652
Data columns (total 53 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  object 
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_cnt            int64  
 14  card_2_txn_cnt            int64  
 15  card_3_txn_cnt            int64  
 16  card_4_txn_cnt            int64  
 17  card_5_txn_cnt            int64  
 18  card_6_txn_cnt            int64  
 19  card_7_txn_cnt            int64  
 20  card_8_

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10321418,45,3,3891.965283,3,0,0,0,1.00,0.00,0.0,0.0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1
1,1,10414574,15,2,10616.561549,2,0,0,0,1.00,0.00,0.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1
2,1,10134567,48,2,23527.655416,0,2,0,0,0.00,1.00,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1
3,1,10001003,48,9,17751.558260,7,2,0,0,0.75,0.25,0.0,0.0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1
4,1,10267183,2,1,21701.307598,1,0,0,0,1.00,0.00,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0


In [4]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

(500000, 1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   chid    500000 non-null  int64
dtypes: int64(1)
memory usage: 3.8 MB
None


,chid
0,10128239
1,10077943
2,10277876
3,10364842
4,10392717


In [5]:
train = train.fillna(0)

In [6]:
# 把shop_tag的other取代為49
train['shop_tag']= train['shop_tag'].replace('other','49')

In [7]:
# 對txn_amt、slam做Normalization，因為小數位數過大會報錯
train['txn_amt'] = (train['txn_amt'] - train['txn_amt'].min())/\
                            (train['txn_amt'].max() - train['txn_amt'].min())
train['slam'] = (train['slam'] - train['slam'].min())/\
                            (train['slam'].max() - train['slam'].min())

In [8]:
# 對卡片消費次數做加總，並刪掉個別消費次數
train["sum_card_txn_cnt"] = train["card_1_txn_cnt"] + train["card_2_txn_cnt"] + train["card_3_txn_cnt"] + train["card_4_txn_cnt"] + train["card_5_txn_cnt"] + train["card_6_txn_cnt"] + train["card_7_txn_cnt"] + train["card_8_txn_cnt"] + train["card_9_txn_cnt"] + train["card_10_txn_cnt"] + train["card_11_txn_cnt"] + train["card_12_txn_cnt"] + train["card_13_txn_cnt"] + train["card_14_txn_cnt"] + train["card_other_txn_cnt"]
train = train.drop(columns = ['card_1_txn_cnt','card_2_txn_cnt','card_3_txn_cnt','card_4_txn_cnt','card_5_txn_cnt','card_6_txn_cnt','card_7_txn_cnt','card_8_txn_cnt','card_9_txn_cnt','card_10_txn_cnt','card_11_txn_cnt','card_12_txn_cnt','card_13_txn_cnt','card_14_txn_cnt','card_other_txn_cnt'])

In [9]:
SelectedFeatures = ['dt', 'chid', 'shop_tag', 'txn_cnt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'sum_card_txn_cnt',
       'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 
       'gender_code', 'age']
train = train[SelectedFeatures]

In [10]:
train = pd.get_dummies(train, columns=['dt'], drop_first=True, prefix='month')

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [12]:
target_column_train = ['shop_tag']
_column_train = ['chid']
predictors_train = list(set(list(train.columns))-set(target_column_train)-set(_column_train))
predictors_train

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

(26380522, 41)
(26380522, 1)


In [13]:
# target_column_test = ['shop_tag'] 
# _column_test = ['chid']
# predictors_test = list(set(list(test.columns))-set(target_column_test)-set(_column_test))

# X_test = test[predictors_test].values
# y_test = test[target_column_test].values

# print(X_test.shape)
# print(y_test.shape)

In [ ]:
# 建立 XGBClassifier 模型
from xgboost import XGBClassifier
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)

# 使用訓練資料訓練模型
xgboostModel.fit(X_train, y_train)

/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to 

In [ ]:
pre_Train = predict_test.drop(columns = ['shop_tag', 'chid'], axis=1, inplace=True)

In [ ]:
# 模型預測
predictions = xgboostModel.predict_proba(pre_Train)

In [ ]:
best_predictions = np.argsort(predictions)[::-1]
best_predictions[0]

In [ ]:
best_predictions_final = np.zeros((len(best_predictions), 3), dtype=int)

In [ ]:
for a in range(len(best_predictions)):
    sum = 0
    for i in range(49): 
        if (best_predictions[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,39,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_predictions[a, i])
            sum += 1 
        if sum == 3: 
            break

In [ ]:
best_predictions_final

In [ ]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [ ]:
filename = 'submission.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)